In [1]:
import requests
import pandas as pd
import os
import math
from bs4 import BeautifulSoup
import re
import numpy as np
import time
import random
import json
from pathlib import Path

# self-definition function
from func4crawl import src02 

In [2]:
path_cd = os.getcwd()
path_root = Path(os.getcwd()).parent
path_data = os.path.join(path_root, "./data")

# configure the folder path for data stored
b = range(0,10)
path_category = [f'{path_data}/category_{i:02}' for i in b]
path_metadata = [os.path.join(path_data, "./metadata")]

---

# target website 02

[Download Free Sound Effects & Royalty Free Music | ZapSplat](https://www.zapsplat.com/)

網站觀察與動作:

- 沒有 JS 渲染 (javascript render): 用 Chrome Extension - Quick Javascript switch 檢查首頁可知。
- 不是AJAX類型網頁: 滑鼠右鍵檢視搜尋結果的網頁原始碼時，可以找到網頁上所顯示的內容，由此可知。
- 沒有API，觀察 source page 
  - 可以抓到 href
- 觀察 url
  - 一頁10筆
  - `https://www.zapsplat.com/page/1/?s=dog+bark`

<!--
- 要做進一步的 filter 
  - keyword
  - duration
-->

--

不知道 filter 的機制，但是感覺比 source 01(BBC) 合理。  
e.g. dog barking 搜尋結果比 dog bark 少 ---> :)

---

**作法:** 

- session 抓取網頁原始碼
- 抓出 href
- 下載 .mp3
- 換頁

--

**Conclusion:**

- keyword
- duration 無資訊，除非播放 AJAX 頁面動態產生。
- 音檔比較準，所以不做tag的篩選，但是metadata裡面仍保有該資訊。

---

## Barking


In [3]:
# 設定關鍵字
query_kws = ['dog+bark', 'dog+barking']

# 設定存檔母路徑
path_category_save = path_category[0]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [4]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

dog+bark
125
----- https://www.zapsplat.com/page/1/?s=dog+bark
----- https://www.zapsplat.com/page/2/?s=dog+bark
----- https://www.zapsplat.com/page/3/?s=dog+bark
----- https://www.zapsplat.com/page/4/?s=dog+bark
----- https://www.zapsplat.com/page/5/?s=dog+bark
----- https://www.zapsplat.com/page/6/?s=dog+bark
----- https://www.zapsplat.com/page/7/?s=dog+bark
----- https://www.zapsplat.com/page/8/?s=dog+bark
----- https://www.zapsplat.com/page/9/?s=dog+bark
----- https://www.zapsplat.com/page/10/?s=dog+bark
----- https://www.zapsplat.com/page/11/?s=dog+bark
----- https://www.zapsplat.com/page/12/?s=dog+bark
----- https://www.zapsplat.com/page/13/?s=dog+bark
dog+barking
147
----- https://www.zapsplat.com/page/1/?s=dog+barking
----- https://www.zapsplat.com/page/2/?s=dog+barking
----- https://www.zapsplat.com/page/3/?s=dog+barking
----- https://www.zapsplat.com/page/4/?s=dog+barking
----- https://www.zapsplat.com/page/5/?s=dog+barking
----- https://www.zapsplat.com/page/6/?s=dog+barking

In [5]:
a.close()
session.close()

In [6]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 272
unique # of posts:	 247


In [7]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [8]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-000.mp3 	 https://www.zapsplat.com/music/2-dogs-barking-small-1/
1 	 s02-001.mp3 	 https://www.zapsplat.com/music/2-dogs-barking-small-2/
2 	 s02-002.mp3 	 https://www.zapsplat.com/music/2-dogs-barking-small-3/
3 	 s02-003.mp3 	 https://www.zapsplat.com/music/2-dogs-barking-small-4/
4 	 s02-004.mp3 	 https://www.zapsplat.com/music/a-dog-barking-and-snarling-angrily/
5 	 s02-005.mp3 	 https://www.zapsplat.com/music/a-dog-barking-and-snarling-scared/
6 	 s02-006.mp3 	 https://www.zapsplat.com/music/a-dog-barking-and-snarling-dog-tag-ringing/
7 	 s02-007.mp3 	 https://www.zapsplat.com/music/aggressive-dog-barking/
8 	 s02-008.mp3 	 https://www.zapsplat.com/music/ambiance-church-bells-wind-dog-bark-people-spain/
9 	 s02-009.mp3 	 https://www.zapsplat.com/music/angry-dog-single-growl-whimper-bark/
10 	 s02-010.mp3 	 https://www.zapsplat.com/music/cartoon-dog-growl-or-bark/
11 	 s02-011.mp3 	 https://www.zapsplat.com/music/cartoon-dog-growls-ruff-bark/
12 	 s02-012.mp3 	 https://www.

91 	 s02-091.mp3 	 https://www.zapsplat.com/music/large-doberman-agitated-aggressive-snarl-outdoor/
92 	 s02-092.mp3 	 https://www.zapsplat.com/music/large-pack-of-dogs-various-breeds-and-sizes-barking-close-up-1/
93 	 s02-093.mp3 	 https://www.zapsplat.com/music/large-pack-of-dogs-various-breeds-and-sizes-barking-close-up-2/
94 	 s02-094.mp3 	 https://www.zapsplat.com/music/medium-sized-dog-barking-5-times-fast-slight-distance/
95 	 s02-095.mp3 	 https://www.zapsplat.com/music/medium-sized-dog-barking-med-close-up/
96 	 s02-096.mp3 	 https://www.zapsplat.com/music/medium-sized-dog-barking-med-close-up-long-version/
97 	 s02-097.mp3 	 https://www.zapsplat.com/music/medium-sized-dog-barking-slight-distance-away/
98 	 s02-098.mp3 	 https://www.zapsplat.com/music/medium-sized-dog-growl-into-a-bark-med-close-up-1/
99 	 s02-099.mp3 	 https://www.zapsplat.com/music/medium-sized-dog-growl-into-a-bark-med-close-up-2/
100 	 s02-100.mp3 	 https://www.zapsplat.com/music/medium-sized-dog-single-ba

182 	 s02-182.mp3 	 https://www.zapsplat.com/music/dog-puppy-barking-several-weeks-old-ridgeback-cross-bullmastiff-7/
183 	 s02-183.mp3 	 https://www.zapsplat.com/music/dog-puppy-barking-several-weeks-old-ridgeback-cross-bullmastiff-8/
184 	 s02-184.mp3 	 https://www.zapsplat.com/music/dog-puppy-barking-several-weeks-old-ridgeback-cross-bullmastiff-9/
185 	 s02-185.mp3 	 https://www.zapsplat.com/music/dog-puppy-single-bark-several-weeks-old-ridgeback-cross-bullmastiff-1/
186 	 s02-186.mp3 	 https://www.zapsplat.com/music/dog-puppy-single-bark-several-weeks-old-ridgeback-cross-bullmastiff-10/
187 	 s02-187.mp3 	 https://www.zapsplat.com/music/dog-puppy-single-bark-several-weeks-old-ridgeback-cross-bullmastiff-11/
188 	 s02-188.mp3 	 https://www.zapsplat.com/music/dog-puppy-single-bark-several-weeks-old-ridgeback-cross-bullmastiff-2/
189 	 s02-189.mp3 	 https://www.zapsplat.com/music/dog-puppy-single-bark-several-weeks-old-ridgeback-cross-bullmastiff-3/
190 	 s02-190.mp3 	 https://www.za

#### save metadata

In [9]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [10]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## Howling

In [11]:
# 設定關鍵字
query_kws = ['dog+howl', 'dog+howling']

# 設定存檔母路徑
path_category_save = path_category[1]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [12]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

dog+howl
1
----- https://www.zapsplat.com/page/1/?s=dog+howl
dog+howling
1
----- https://www.zapsplat.com/page/1/?s=dog+howling


In [13]:
a.close()
session.close()

In [14]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 2
unique # of posts:	 2


In [15]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [16]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-0.mp3 	 https://www.zapsplat.com/music/cartoon-dog-howl/
1 	 s02-1.mp3 	 https://www.zapsplat.com/music/dog-wolf-howling-4-versions/
Wall time: 2.85 s


#### save metadata

In [17]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [18]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## Crying

---

## CO_Smoke

In [19]:
# 設定關鍵字
query_kws = ['smoke+detector', ' fire+alarm+beep', 'alarm+beep', 'detector', 'detector+beep']

# 設定存檔母路徑
path_category_save = path_category[3]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [20]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

smoke+detector
6
----- https://www.zapsplat.com/page/1/?s=smoke+detector
 fire+alarm+beep
0
alarm+beep
40
----- https://www.zapsplat.com/page/1/?s=alarm+beep
----- https://www.zapsplat.com/page/2/?s=alarm+beep
----- https://www.zapsplat.com/page/3/?s=alarm+beep
----- https://www.zapsplat.com/page/4/?s=alarm+beep
detector
8
----- https://www.zapsplat.com/page/1/?s=detector
detector+beep
5
----- https://www.zapsplat.com/page/1/?s=detector+beep


In [21]:
a.close()
session.close()

In [22]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 59
unique # of posts:	 57


In [23]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [24]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-00.mp3 	 https://www.zapsplat.com/music/alarm-domestic-smoke-detector/
1 	 s02-01.mp3 	 https://www.zapsplat.com/music/domestic-smoke-detector-unit-place-down-on-surface-1/
2 	 s02-02.mp3 	 https://www.zapsplat.com/music/domestic-smoke-detector-unit-place-down-on-surface-2/
3 	 s02-03.mp3 	 https://www.zapsplat.com/music/smoke-detector-fire-alarm-beep-tone-1/
4 	 s02-04.mp3 	 https://www.zapsplat.com/music/smoke-detector-fire-alarm-beep-tone-2/
5 	 s02-05.mp3 	 https://www.zapsplat.com/music/smoke-detector-fire-alarm-beep-tone-3/
6 	 s02-06.mp3 	 https://www.zapsplat.com/music/alarm-clock-beep-tone/
7 	 s02-07.mp3 	 https://www.zapsplat.com/music/alarm-clock-beeps-digital-1/
8 	 s02-08.mp3 	 https://www.zapsplat.com/music/alarm-clock-beeps-digital-2/
9 	 s02-09.mp3 	 https://www.zapsplat.com/music/alarm-clock-beeps-digital-3/
10 	 s02-10.mp3 	 https://www.zapsplat.com/music/alarm-clock-going-off-digital-beeps-1/
11 	 s02-11.mp3 	 https://www.zapsplat.com/music/alarm-clock-going

#### save metadata

In [25]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [26]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## GlassBreaking

In [77]:
# 設定關鍵字
query_kws = ['glass+break', 'broken+glass']

# 設定存檔母路徑
path_category_save = path_category[4]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [78]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

glass+break
156
----- https://www.zapsplat.com/page/1/?s=glass+break
----- https://www.zapsplat.com/page/2/?s=glass+break
----- https://www.zapsplat.com/page/3/?s=glass+break
----- https://www.zapsplat.com/page/4/?s=glass+break
----- https://www.zapsplat.com/page/5/?s=glass+break
----- https://www.zapsplat.com/page/6/?s=glass+break
----- https://www.zapsplat.com/page/7/?s=glass+break
----- https://www.zapsplat.com/page/8/?s=glass+break
----- https://www.zapsplat.com/page/9/?s=glass+break
----- https://www.zapsplat.com/page/10/?s=glass+break
----- https://www.zapsplat.com/page/11/?s=glass+break
----- https://www.zapsplat.com/page/12/?s=glass+break
----- https://www.zapsplat.com/page/13/?s=glass+break
----- https://www.zapsplat.com/page/14/?s=glass+break
----- https://www.zapsplat.com/page/15/?s=glass+break
----- https://www.zapsplat.com/page/16/?s=glass+break
broken+glass
269
----- https://www.zapsplat.com/page/1/?s=broken+glass
----- https://www.zapsplat.com/page/2/?s=broken+glass
----

In [79]:
a.close()
session.close()

In [80]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 425
unique # of posts:	 423


In [81]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [82]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-000.mp3 	 https://www.zapsplat.com/music/broken-glass-impact-slam-hard-hit-1/
1 	 s02-001.mp3 	 https://www.zapsplat.com/music/broken-glass-impact-slam-hard-hit-2/
2 	 s02-002.mp3 	 https://www.zapsplat.com/music/broken-glass-impact-slam-hard-hit-3/
3 	 s02-003.mp3 	 https://www.zapsplat.com/music/bullet-hit-glass-bottle-smash-1/
4 	 s02-004.mp3 	 https://www.zapsplat.com/music/bullet-hit-glass-bottle-smash-2/
5 	 s02-005.mp3 	 https://www.zapsplat.com/music/bullet-hit-glass-bottle-smash-3/
6 	 s02-006.mp3 	 https://www.zapsplat.com/music/explosion-large-with-glass-breaking-and-other-debris-1/
7 	 s02-007.mp3 	 https://www.zapsplat.com/music/explosion-large-with-glass-breaking-and-other-debris-2/
8 	 s02-008.mp3 	 https://www.zapsplat.com/music/fast-whoosh-or-swipe-into-glass-smash-version-1/
9 	 s02-009.mp3 	 https://www.zapsplat.com/music/fast-whoosh-or-swipe-into-glass-smash-version-2/
10 	 s02-010.mp3 	 https://www.zapsplat.com/music/full-glass-beer-bottle-drop-onto-concret

87 	 s02-087.mp3 	 https://www.zapsplat.com/music/impact-cardboard-box-containing-broken-glass-debris-drop-on-floor/
88 	 s02-088.mp3 	 https://www.zapsplat.com/music/impact-glass-bottle-drop-onto-broken-glass-debris-1/
89 	 s02-089.mp3 	 https://www.zapsplat.com/music/impact-glass-bottle-drop-onto-broken-glass-debris-2/
90 	 s02-090.mp3 	 https://www.zapsplat.com/music/impact-glass-bottle-hit-and-break-with-hammer-1/
91 	 s02-091.mp3 	 https://www.zapsplat.com/music/impact-glass-bottle-hit-and-break-with-hammer-2/
92 	 s02-092.mp3 	 https://www.zapsplat.com/music/impact-glass-bottle-hit-and-break-with-hammer-3/
93 	 s02-093.mp3 	 https://www.zapsplat.com/music/impact-glass-bottle-hit-broken-glass-debris-inside-cardboard-box-multiple-times/
94 	 s02-094.mp3 	 https://www.zapsplat.com/music/impact-glass-bottle-throw-onto-broken-glass-debris/
95 	 s02-095.mp3 	 https://www.zapsplat.com/music/impact-glass-jar-hit-and-break-with-hammer/
96 	 s02-096.mp3 	 https://www.zapsplat.com/music/imp

168 	 s02-168.mp3 	 https://www.zapsplat.com/music/broken-glass-bottles-x2-inside-plastic-bag-drop-5/
169 	 s02-169.mp3 	 https://www.zapsplat.com/music/broken-glass-bottles-x2-inside-plastic-bag-drop-6/
170 	 s02-170.mp3 	 https://www.zapsplat.com/music/broken-glass-bottles-x2-inside-plastic-bag-drop-7/
171 	 s02-171.mp3 	 https://www.zapsplat.com/music/broken-glass-impact-slam-hard-hit-1/
172 	 s02-172.mp3 	 https://www.zapsplat.com/music/broken-glass-impact-slam-hard-hit-2/
173 	 s02-173.mp3 	 https://www.zapsplat.com/music/broken-glass-impact-slam-hard-hit-3/
174 	 s02-174.mp3 	 https://www.zapsplat.com/music/broken-glass-in-cardboard-box-drop-down-on-ground-1/
175 	 s02-175.mp3 	 https://www.zapsplat.com/music/broken-glass-in-cardboard-box-drop-down-on-ground-2/
176 	 s02-176.mp3 	 https://www.zapsplat.com/music/broken-glass-in-cardboard-box-drop-down-on-ground-3/
177 	 s02-177.mp3 	 https://www.zapsplat.com/music/broken-glass-in-cardboard-box-movements-1/
178 	 s02-178.mp3 	 http

248 	 s02-248.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-roll-on-carpet-3/
249 	 s02-249.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-roll-on-concrete-1/
250 	 s02-250.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-roll-on-concrete-2/
251 	 s02-251.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-shake-1/
252 	 s02-252.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-shake-2/
253 	 s02-253.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-shake-3/
254 	 s02-254.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-shake-4/
255 	 s02-255.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-shake-5/
256 	 s02-256.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-glass-inside-glass-jar-shake-6/
257 	 s02-257.mp3 	 https://www.zapsplat.com/music/broken-pieces-of-g

323 	 s02-323.mp3 	 https://www.zapsplat.com/music/piece-of-thick-broken-glass-movement-on-top-of-another-piece-4/
324 	 s02-324.mp3 	 https://www.zapsplat.com/music/rummage-through-broken-pieces-of-glass-in-metal-container-1/
325 	 s02-325.mp3 	 https://www.zapsplat.com/music/rummage-through-broken-pieces-of-glass-in-metal-container-2/
326 	 s02-326.mp3 	 https://www.zapsplat.com/music/rummage-through-broken-pieces-of-glass-in-metal-container-3/
327 	 s02-327.mp3 	 https://www.zapsplat.com/music/rummage-through-broken-pieces-of-glass-in-metal-container-4/
328 	 s02-328.mp3 	 https://www.zapsplat.com/music/rummage-through-broken-pieces-of-glass-in-metal-container-5/
329 	 s02-329.mp3 	 https://www.zapsplat.com/music/rummage-through-broken-pieces-of-glass-in-metal-container-6/
330 	 s02-330.mp3 	 https://www.zapsplat.com/music/several-glass-shards-from-broken-picture-frame-drop-doesnt-break/
331 	 s02-331.mp3 	 https://www.zapsplat.com/music/sharp-edge-of-thick-broken-piece-of-glass-scr

396 	 s02-396.mp3 	 https://www.zapsplat.com/music/small-pieces-of-broken-shattered-glass-throw-onto-thousands-more-in-plastic-container/
397 	 s02-397.mp3 	 https://www.zapsplat.com/music/small-shard-from-broken-lightbulb-impact-hit/
398 	 s02-398.mp3 	 https://www.zapsplat.com/music/small-shard-of-broken-glass-bottle-throw-in-bin-impact-break-smash-1/
399 	 s02-399.mp3 	 https://www.zapsplat.com/music/small-shard-of-broken-glass-bottle-throw-in-bin-impact-break-smash-2/
400 	 s02-400.mp3 	 https://www.zapsplat.com/music/small-shard-of-broken-glass-bottle-throw-in-bin-impact-break-smash-3/
401 	 s02-401.mp3 	 https://www.zapsplat.com/music/small-shard-of-broken-glass-bottle-throw-in-bin-impact-break-smash-4/
402 	 s02-402.mp3 	 https://www.zapsplat.com/music/small-shard-of-broken-glass-bottle-throw-in-bin-impact-break-smash-5/
403 	 s02-403.mp3 	 https://www.zapsplat.com/music/small-shard-of-broken-glass-bottle-throw-in-bin-impact-break-smash-6/
404 	 s02-404.mp3 	 https://www.zapspla

#### save metadata

In [83]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [84]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## Other

Other, Vacuum, Blender, Electrics, Cat, Dishes

In [35]:
# 設定關鍵字
query_kws = ['meow', 'cat+voice', 'vacuum+cleaner', 'vacuum', 'blender start', 'blender', 'NutriBullet', 'toy+noise', 'electrical+current', 'phone+ring', 'telephone+ring', 'dishes']

# 設定存檔母路徑
path_category_save = path_category[5]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [36]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

meow
18
----- https://www.zapsplat.com/page/1/?s=meow
----- https://www.zapsplat.com/page/2/?s=meow
cat+voice
2
----- https://www.zapsplat.com/page/1/?s=cat+voice
vacuum+cleaner
163
----- https://www.zapsplat.com/page/1/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/2/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/3/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/4/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/5/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/6/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/7/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/8/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/9/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/10/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/11/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/12/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/13/?s=vacuum+cleaner
----- https://www.zapsplat.com/page/14/?s=vacuum+cleaner
----- https://www.zapsplat.co

In [37]:
a.close()
session.close()

In [38]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 567
unique # of posts:	 452


In [39]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [40]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-000.mp3 	 https://www.zapsplat.com/music/cartoon-cat-meow/
1 	 s02-001.mp3 	 https://www.zapsplat.com/music/cat-meow-1/
2 	 s02-002.mp3 	 https://www.zapsplat.com/music/cat-meow-2/
3 	 s02-003.mp3 	 https://www.zapsplat.com/music/cat-meow-3/
4 	 s02-004.mp3 	 https://www.zapsplat.com/music/cat-meow-hungry/
5 	 s02-005.mp3 	 https://www.zapsplat.com/music/cat-meow-hungry-begging-for-food/
6 	 s02-006.mp3 	 https://www.zapsplat.com/music/cat-meowing/
7 	 s02-007.mp3 	 https://www.zapsplat.com/music/cat-kitten-meow-1/
8 	 s02-008.mp3 	 https://www.zapsplat.com/music/cat-kitten-meow-2/
9 	 s02-009.mp3 	 https://www.zapsplat.com/music/cat-kitten-meow-3/
10 	 s02-010.mp3 	 https://www.zapsplat.com/music/cat-kitten-meow-4/
11 	 s02-011.mp3 	 https://www.zapsplat.com/music/cat-kitten-meow-5/
12 	 s02-012.mp3 	 https://www.zapsplat.com/music/cat-kitten-meow-6/
13 	 s02-013.mp3 	 https://www.zapsplat.com/music/cat-kitten-meow-7/
14 	 s02-014.mp3 	 https://www.zapsplat.com/music/cat-kitte

81 	 s02-081.mp3 	 https://www.zapsplat.com/music/robotic-vacuum-cleaner-vacuuming-pass-by-on-wood-floor/
82 	 s02-082.mp3 	 https://www.zapsplat.com/music/small-handheld-vacuum-cleaner-set-down-on-hard-ground-1/
83 	 s02-083.mp3 	 https://www.zapsplat.com/music/small-handheld-vacuum-cleaner-set-down-on-hard-ground-2/
84 	 s02-084.mp3 	 https://www.zapsplat.com/music/small-handheld-vacuum-cleaner-short-burst/
85 	 s02-085.mp3 	 https://www.zapsplat.com/music/small-handheld-vacuum-cleaner-turn-on-burst-then-off-1/
86 	 s02-086.mp3 	 https://www.zapsplat.com/music/small-handheld-vacuum-cleaner-turn-on-burst-then-off-2/
87 	 s02-087.mp3 	 https://www.zapsplat.com/music/small-handheld-vacuum-cleaner-turn-on-burst-then-off-3/
88 	 s02-088.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-bagless-dyson-switches-on-and-operates-includes-room-reverb-medium-sized-furnished-living-room/
89 	 s02-089.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-bagless-type-cylinder-put-on-unit/
90 	 s02

159 	 s02-159.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-vacuuming-close-up/
160 	 s02-160.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-vacuuming-fabric-sofa-couch-recorded-close-up-long-version/
161 	 s02-161.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-vacuuming-fabric-sofa-couch-recorded-close-up-short-version/
162 	 s02-162.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-vacuuming-out-car-nozzle-end-suction-movements-1/
163 	 s02-163.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-vacuuming-out-car-nozzle-end-suction-movements-2/
164 	 s02-164.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-vacuuming-up-debris-bits-crumbs-from-carpet/
165 	 s02-165.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-very-short-on-and-off-burst-false-start-1/
166 	 s02-166.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-very-short-on-and-off-burst-false-start-2/
167 	 s02-167.mp3 	 https://www.zapsplat.com/music/vacuum-cleaner-wide-roller-suction-nozzle-roll-on

234 	 s02-234.mp3 	 https://www.zapsplat.com/music/handheld-vacuum-cleaner-dustbuster-place-on-charging-dock/
235 	 s02-235.mp3 	 https://www.zapsplat.com/music/handheld-vacuum-cleaner-dustbuster-remove-from-charging-dock/
236 	 s02-236.mp3 	 https://www.zapsplat.com/music/handheld-vacuum-cleaner-on-run-short-burst-1/
237 	 s02-237.mp3 	 https://www.zapsplat.com/music/handheld-vacuum-cleaner-on-run-short-burst-2/
238 	 s02-238.mp3 	 https://www.zapsplat.com/music/hoover-spirit-vacuum-cleaner-fall-over-softly/
239 	 s02-239.mp3 	 https://www.zapsplat.com/music/hoover-spirit-vacuum-cleaner-push-along-floor-while-off/
240 	 s02-240.mp3 	 https://www.zapsplat.com/music/robot-vacuum-operating-vacuuming-tiled-floor-room-ambience-perspective-1/
241 	 s02-241.mp3 	 https://www.zapsplat.com/music/robot-vacuum-operating-vacuuming-tiled-floor-room-ambience-perspective-2/
242 	 s02-242.mp3 	 https://www.zapsplat.com/music/robotic-vacuum-approach-then-vacuum-close-by-on-tiled-floor/
243 	 s02-243.m

310 	 s02-310.mp3 	 https://www.zapsplat.com/music/electric-food-blender-turn-on-run-then-off-very-short-burst-liquid-inside-2/
311 	 s02-311.mp3 	 https://www.zapsplat.com/music/electric-food-mixer-runs-slow-then-turned-up-to-fast-an-back-to-slow-2/
312 	 s02-312.mp3 	 https://www.zapsplat.com/music/electric-hand-blender-operating-two-bursts/
313 	 s02-313.mp3 	 https://www.zapsplat.com/music/food-blender-button-press/
314 	 s02-314.mp3 	 https://www.zapsplat.com/music/food-blender-speed-control-dial-turn-1/
315 	 s02-315.mp3 	 https://www.zapsplat.com/music/food-blender-speed-control-dial-turn-2/
316 	 s02-316.mp3 	 https://www.zapsplat.com/music/food-blender-speed-control-dial-turn-3/
317 	 s02-317.mp3 	 https://www.zapsplat.com/music/food-blender-speed-control-dial-turn-4/
318 	 s02-318.mp3 	 https://www.zapsplat.com/music/food-blender-speed-control-dial-turn-5/
319 	 s02-319.mp3 	 https://www.zapsplat.com/music/food-mixer-min-master-operating-2/
320 	 s02-320.mp3 	 https://www.zap

391 	 s02-391.mp3 	 https://www.zapsplat.com/music/telephone-old-style-rotary-dial-dialling-number-7-2/
392 	 s02-392.mp3 	 https://www.zapsplat.com/music/telephone-old-style-rotary-dial-dialling-number-8-2/
393 	 s02-393.mp3 	 https://www.zapsplat.com/music/telephone-old-style-rotary-dial-dialling-number-9-2/
394 	 s02-394.mp3 	 https://www.zapsplat.com/music/telephone-ringing-tone-uk-in-ear-piece-perspective-2/
395 	 s02-395.mp3 	 https://www.zapsplat.com/music/telephone-uk-ringing-tone/
396 	 s02-396.mp3 	 https://www.zapsplat.com/music/uk-mobile-telephone-ring-device-speaker-ringing-tone-long-version/
397 	 s02-397.mp3 	 https://www.zapsplat.com/music/uk-mobile-telephone-ring-device-speaker-ringing-tone-medium-version/
398 	 s02-398.mp3 	 https://www.zapsplat.com/music/uk-mobile-telephone-ring-device-speaker-ringing-tone-short-version/
399 	 s02-399.mp3 	 https://www.zapsplat.com/music/vintage-telephone-australian-standard-ringing-siemens-h70-1968/
400 	 s02-400.mp3 	 https://www.z

#### save metadata

In [41]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [44]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f, )
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## Doorbell

In [45]:
# 設定關鍵字
query_kws = ['doorbell', 'door+ring']

# 設定存檔母路徑
path_category_save = path_category[6]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [46]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

doorbell
40
----- https://www.zapsplat.com/page/1/?s=doorbell
----- https://www.zapsplat.com/page/2/?s=doorbell
----- https://www.zapsplat.com/page/3/?s=doorbell
----- https://www.zapsplat.com/page/4/?s=doorbell
door+ring
33
----- https://www.zapsplat.com/page/1/?s=door+ring
----- https://www.zapsplat.com/page/2/?s=door+ring
----- https://www.zapsplat.com/page/3/?s=door+ring
----- https://www.zapsplat.com/page/4/?s=door+ring


In [47]:
a.close()
session.close()

In [48]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 73
unique # of posts:	 68


In [49]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [50]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-00.mp3 	 https://www.zapsplat.com/music/classic-ding-dong-doorbell-chime-1/
1 	 s02-01.mp3 	 https://www.zapsplat.com/music/classic-ding-dong-doorbell-chime-2/
2 	 s02-02.mp3 	 https://www.zapsplat.com/music/classic-ding-dong-doorbell-chime-3/
3 	 s02-03.mp3 	 https://www.zapsplat.com/music/door-bell-ring-internal-recording/
4 	 s02-04.mp3 	 https://www.zapsplat.com/music/doorbell-1/
5 	 s02-05.mp3 	 https://www.zapsplat.com/music/doorbell-2/
6 	 s02-06.mp3 	 https://www.zapsplat.com/music/doorbell-3/
7 	 s02-07.mp3 	 https://www.zapsplat.com/music/doorbell-carillon-chime/
8 	 s02-08.mp3 	 https://www.zapsplat.com/music/doorbell-chime-1/
9 	 s02-09.mp3 	 https://www.zapsplat.com/music/doorbell-chime-2/
10 	 s02-10.mp3 	 https://www.zapsplat.com/music/doorbell-chime-3/
11 	 s02-11.mp3 	 https://www.zapsplat.com/music/doorbell-chime-4/
12 	 s02-12.mp3 	 https://www.zapsplat.com/music/doorbell-chime-fast-impatient-repetitive-presses-1/
13 	 s02-13.mp3 	 https://www.zapsplat.com/mu

#### save metadata

In [51]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [52]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## Bird

In [53]:
# 設定關鍵字
query_kws = ['bird', 'bird+chirping']

# 設定存檔母路徑
path_category_save = path_category[7]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [54]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

bird
244
----- https://www.zapsplat.com/page/1/?s=bird
----- https://www.zapsplat.com/page/2/?s=bird
----- https://www.zapsplat.com/page/3/?s=bird
----- https://www.zapsplat.com/page/4/?s=bird
----- https://www.zapsplat.com/page/5/?s=bird
----- https://www.zapsplat.com/page/6/?s=bird
----- https://www.zapsplat.com/page/7/?s=bird
----- https://www.zapsplat.com/page/8/?s=bird
----- https://www.zapsplat.com/page/9/?s=bird
----- https://www.zapsplat.com/page/10/?s=bird
----- https://www.zapsplat.com/page/11/?s=bird
----- https://www.zapsplat.com/page/12/?s=bird
----- https://www.zapsplat.com/page/13/?s=bird
----- https://www.zapsplat.com/page/14/?s=bird
----- https://www.zapsplat.com/page/15/?s=bird
----- https://www.zapsplat.com/page/16/?s=bird
----- https://www.zapsplat.com/page/17/?s=bird
----- https://www.zapsplat.com/page/18/?s=bird
----- https://www.zapsplat.com/page/19/?s=bird
----- https://www.zapsplat.com/page/20/?s=bird
----- https://www.zapsplat.com/page/21/?s=bird
----- https:/

In [55]:
a.close()
session.close()

In [56]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 301
unique # of posts:	 296


In [57]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [58]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-000.mp3 	 https://www.zapsplat.com/music/a-flute-imitating-a-singing-bird/
1 	 s02-001.mp3 	 https://www.zapsplat.com/music/amazon-jungle-ambience-bird-calls-medium-and-distant-perspectives-loud-insect-hum-medium-distant-tree-cracks-and-falls-into-water-near-tail/
2 	 s02-002.mp3 	 https://www.zapsplat.com/music/ambience-ambiance-ambient-forest-nature-florida-woods-wild-grass-outdoor-trees-day-dusk-loop-seamless-looping-bird-chirp-chirping-squeak-squawk/
3 	 s02-003.mp3 	 https://www.zapsplat.com/music/australia-forest-ambience-short-unidentified-bird-call-wind-in-trees-winter-queensland/
4 	 s02-004.mp3 	 https://www.zapsplat.com/music/baby-bird-chick-or-other-hatchling-emerge-from-egg-1/
5 	 s02-005.mp3 	 https://www.zapsplat.com/music/baby-bird-chick-or-other-hatchling-emerge-from-egg-2/
6 	 s02-006.mp3 	 https://www.zapsplat.com/music/baby-bird-chick-hatchling-call-chirp/
7 	 s02-007.mp3 	 https://www.zapsplat.com/music/baby-bird-nestling-tweet-in-nest-1/
8 	 s02-008.mp3 	 

90 	 s02-090.mp3 	 https://www.zapsplat.com/music/bird-peewee-call-australia-2/
91 	 s02-091.mp3 	 https://www.zapsplat.com/music/bird-peewee-call-australia-3/
92 	 s02-092.mp3 	 https://www.zapsplat.com/music/bird-pigeon-feet-claws-walking-on-wood-surface/
93 	 s02-093.mp3 	 https://www.zapsplat.com/music/bird-red-tail-hawk-red-tail-hawk-screams/
94 	 s02-094.mp3 	 https://www.zapsplat.com/music/bird-red-billed-hornbill-tochus-erythrorhynchus-insects/
95 	 s02-095.mp3 	 https://www.zapsplat.com/music/bird-seagull-four-single-cries-close-perspective/
96 	 s02-096.mp3 	 https://www.zapsplat.com/music/bird-single-chirp/
97 	 s02-097.mp3 	 https://www.zapsplat.com/music/bird-sparrow-chirping-and-singing-with-wing-flaps-and-flying-1/
98 	 s02-098.mp3 	 https://www.zapsplat.com/music/bird-sparrow-chirping-and-singing-with-wing-flaps-and-flying-2/
99 	 s02-099.mp3 	 https://www.zapsplat.com/music/bird-spotted-dove-call-australia/
100 	 s02-100.mp3 	 https://www.zapsplat.com/music/bird-starli

173 	 s02-173.mp3 	 https://www.zapsplat.com/music/flapping-wings-of-a-bird-foley-with-leather-gloves-hectic-generic-movement/
174 	 s02-174.mp3 	 https://www.zapsplat.com/music/flapping-wings-of-a-bird-foley-with-leather-gloves-hectic-movement/
175 	 s02-175.mp3 	 https://www.zapsplat.com/music/flapping-wings-of-a-bird-foley-with-leather-gloves-single-flap/
176 	 s02-176.mp3 	 https://www.zapsplat.com/music/flapping-wings-of-a-bird-foley-with-leather-gloves-slow-but-strong-movement/
177 	 s02-177.mp3 	 https://www.zapsplat.com/music/flapping-wings-of-a-bird-foley-with-leather-gloves-soft-subtle-and-slow-movement/
178 	 s02-178.mp3 	 https://www.zapsplat.com/music/flapping-wings-of-a-bird-foley-with-leather-gloves-very-nervous-and-fast-movement/
179 	 s02-179.mp3 	 https://www.zapsplat.com/music/forest-ambience-distant-river-and-planes-some-light-bird-noise/
180 	 s02-180.mp3 	 https://www.zapsplat.com/music/forest-ambience-strong-wind-in-trees-insects-spangled-drongo-bird-call-starts-

241 	 s02-241.mp3 	 https://www.zapsplat.com/music/weed-whacker-operating-cutting-grass-in-neighbours-garden-slight-distance-some-bird-noise/
242 	 s02-242.mp3 	 https://www.zapsplat.com/music/wetland-at-night-ambience-frogs-insects-and-a-heron-bird-mexico-1/
243 	 s02-243.mp3 	 https://www.zapsplat.com/music/wetland-at-night-ambience-frogs-insects-and-a-heron-bird-mexico-2/
244 	 s02-244.mp3 	 https://www.zapsplat.com/music/ambience-ambiance-ambient-forest-nature-florida-woods-wild-grass-outdoor-trees-day-dusk-loop-seamless-looping-bird-chirp-chirping-squeak-squawk/
245 	 s02-245.mp3 	 https://www.zapsplat.com/music/baby-bird-chick-hatchling-call-chirp/
246 	 s02-246.mp3 	 https://www.zapsplat.com/music/baby-bird-nestling-tweet-in-nest-1/
247 	 s02-247.mp3 	 https://www.zapsplat.com/music/baby-bird-nestling-tweet-in-nest-2/
248 	 s02-248.mp3 	 https://www.zapsplat.com/music/baby-birds-hatchlings-tweeting-in-nest-for-food/
249 	 s02-249.mp3 	 https://www.zapsplat.com/music/baby-birds-n

#### save metadata

In [59]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [60]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## Music_Instrument

In [61]:
# 設定關鍵字
query_kws = ['warm+pad', 'piano', 'soft+piano', 'percussion']

# 設定存檔母路徑
path_category_save = path_category[8]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [62]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

warm+pad
9
----- https://www.zapsplat.com/page/1/?s=warm+pad
piano
210
----- https://www.zapsplat.com/page/1/?s=piano
----- https://www.zapsplat.com/page/2/?s=piano
----- https://www.zapsplat.com/page/3/?s=piano
----- https://www.zapsplat.com/page/4/?s=piano
----- https://www.zapsplat.com/page/5/?s=piano
----- https://www.zapsplat.com/page/6/?s=piano
----- https://www.zapsplat.com/page/7/?s=piano
----- https://www.zapsplat.com/page/8/?s=piano
----- https://www.zapsplat.com/page/9/?s=piano
----- https://www.zapsplat.com/page/10/?s=piano
----- https://www.zapsplat.com/page/11/?s=piano
----- https://www.zapsplat.com/page/12/?s=piano
----- https://www.zapsplat.com/page/13/?s=piano
----- https://www.zapsplat.com/page/14/?s=piano
----- https://www.zapsplat.com/page/15/?s=piano
----- https://www.zapsplat.com/page/16/?s=piano
----- https://www.zapsplat.com/page/17/?s=piano
----- https://www.zapsplat.com/page/18/?s=piano
----- https://www.zapsplat.com/page/19/?s=piano
----- https://www.zapsplat

In [63]:
a.close()
session.close()

In [64]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 305
unique # of posts:	 301


In [65]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [66]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-000.mp3 	 https://www.zapsplat.com/music/alert-or-notification-soft-piano-and-warm-pad-1/
1 	 s02-001.mp3 	 https://www.zapsplat.com/music/alert-or-notification-soft-piano-and-warm-pad-2/
2 	 s02-002.mp3 	 https://www.zapsplat.com/music/alert-or-notification-soft-piano-and-warm-pad-3/
3 	 s02-003.mp3 	 https://www.zapsplat.com/music/magic-spell-ascending-metallic-with-warm-pad/
4 	 s02-004.mp3 	 https://www.zapsplat.com/music/multimedia-signature-or-musical-logo-branding-element-warm-soft-marimba-and-pad/
5 	 s02-005.mp3 	 https://www.zapsplat.com/music/musical-game-intro-warm-pad-with-a-high-pitched-mallet-twinkle/
6 	 s02-006.mp3 	 https://www.zapsplat.com/music/musical-game-intro-warm-pad-with-a-wooden-mallet-melody/
7 	 s02-007.mp3 	 https://www.zapsplat.com/music/musical-game-intro-warm-pad-with-a-wooden-mallet-ping/
8 	 s02-008.mp3 	 https://www.zapsplat.com/music/realization-downtempo-track-with-a-serious-feel-repeating-organ-arpeggio-melody-warm-electric-piano-discordan

70 	 s02-070.mp3 	 https://www.zapsplat.com/music/designed-riser-production-element-reversed-piano-dark-and-dramatic/
71 	 s02-071.mp3 	 https://www.zapsplat.com/music/difficult-decisions-melancholy-dark-and-sombre-down-tempo-piano-melody-warm-pads/
72 	 s02-072.mp3 	 https://www.zapsplat.com/music/dream-of-grandeur-positive-motivational-music-with-piano-strings-drums-brass-and-a-building-sense-of-achievement/
73 	 s02-073.mp3 	 https://www.zapsplat.com/music/droplets-of-dew-a-calm-relaxing-and-lullaby-like-ambience-piece-with-piano-guitar-strings-and-flute/
74 	 s02-074.mp3 	 https://www.zapsplat.com/music/enchanting-warm-smooth-easy-listening-jazz-with-piano-rhythm-and-guitar-rhythm/
75 	 s02-075.mp3 	 https://www.zapsplat.com/music/escape-tv-or-film-music-fast-paced-electronic-arpeggio-drives-this-action-piece-with-brass-and-piano-stabs-and-electric-guitar/
76 	 s02-076.mp3 	 https://www.zapsplat.com/music/evening-tide-warm-sincere-music-with-piano-nylon-string-guitar-and-flute-melo

132 	 s02-132.mp3 	 https://www.zapsplat.com/music/kids-game-sound-cheeky-cute-and-fun-electric-piano-bell-tone-notification/
133 	 s02-133.mp3 	 https://www.zapsplat.com/music/kids-game-sound-cheeky-cute-and-fun-electric-piano-bell-tone-positive-1/
134 	 s02-134.mp3 	 https://www.zapsplat.com/music/kids-game-sound-cheeky-cute-and-fun-electric-piano-bell-tone-positive-2/
135 	 s02-135.mp3 	 https://www.zapsplat.com/music/kids-game-sound-cheeky-cute-and-fun-electric-piano-bell-tone-success/
136 	 s02-136.mp3 	 https://www.zapsplat.com/music/last-chance-heavy-weight-tension-fulled-music-bed-with-heavy-kick-drum-and-other-drums-dark-arpeggio-and-piano/
137 	 s02-137.mp3 	 https://www.zapsplat.com/music/lazy-days-chillwave-laid-back-dreamy-groove-with-a-nostalgic-feel-hazy-chords-electric-piano-guitar-and-old-school-drum-loop/
138 	 s02-138.mp3 	 https://www.zapsplat.com/music/lets-start-the-show-energetic-intro-outtro-music-to-podcast-drums-bass-piano-acoustic-and-electric-guitars/
139 	 

197 	 s02-197.mp3 	 https://www.zapsplat.com/music/standing-tall-energetic-music-driven-by-bass-drums-piano-brass-and-strings-conveys-a-sense-of-action-and-success/
198 	 s02-198.mp3 	 https://www.zapsplat.com/music/still-learning-marimba-arpeggio-drives-this-serious-high-brow-music-bed-with-4-4-drums-and-a-light-techno-feel-piano-melody/
199 	 s02-199.mp3 	 https://www.zapsplat.com/music/summer-haze-slow-chill-out-house-track-with-a-modern-pop-feel-warm-piano-chords-underpin-the-track-with-warm-pads-and-a-repetitive-synth-arpeggio/
200 	 s02-200.mp3 	 https://www.zapsplat.com/music/summit-positive-motivational-anthemic-track-with-a-panoramic-feel-starts-with-a-dreamy-choir-piano-chords-and-synth-pads-builds-slowly-into-pounding-drums-into-an-anthemic-climax/
201 	 s02-201.mp3 	 https://www.zapsplat.com/music/swell-whoosh-piano-musical-reveal-impact-boom-hit-deep-heavy-dramatic-ominous-foreboding-eerie-dark-scifi-horror/
202 	 s02-202.mp3 	 https://www.zapsplat.com/music/swell-whoosh-r

255 	 s02-255.mp3 	 https://www.zapsplat.com/music/drum-circle-percussion-bongos-venice-los-angeles-usa-1/
256 	 s02-256.mp3 	 https://www.zapsplat.com/music/drum-circle-percussion-bongos-venice-los-angeles-usa-1-2/
257 	 s02-257.mp3 	 https://www.zapsplat.com/music/drum-circle-percussion-bongos-venice-los-angeles-usa-1-3/
258 	 s02-258.mp3 	 https://www.zapsplat.com/music/gothic-dark-and-tension-building-orchestral-piece-with-strings-wind-instruments-and-percussion-lots-of-drama-and-elements-of-danger/
259 	 s02-259.mp3 	 https://www.zapsplat.com/music/happy-seamless-music-loop-acoustic-lazy-summer-feel-positive-fun-ukulele-bass-drums-and-percussion/
260 	 s02-260.mp3 	 https://www.zapsplat.com/music/happy-seamless-music-loop-acoustic-positive-confident-fun-and-sincere-acoustic-guitars-bass-and-drums-and-percussion/
261 	 s02-261.mp3 	 https://www.zapsplat.com/music/hostage-crisis-dramatic-tense-music-with-orchestral-strings-brass-percussion-and-electric-guitar-gives-the-feel-of-a-evo

#### save metadata

In [67]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [68]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

---

## Laugh_Shout_Scream

In [69]:
# 設定關鍵字
query_kws = ['laugh', 'laughter', 'shouting', 'scream', 'screaming']

# 設定存檔母路徑
path_category_save = path_category[9]

# 設定初始爬取網址的路徑
tmp_url = 'https://www.zapsplat.com/?s='

In [70]:
# crawl data
session = requests.session()
posts = []
for i_kws in query_kws:
    print(i_kws)
    
    # 搜尋結果的筆數
    a = session.get(tmp_url+i_kws)
    soup = BeautifulSoup(a.text,'lxml')
    tmp_x = soup.find('h1', {'class':'page-title'}).text
    tmp_num = re.search('[0-9]+(?= results)', tmp_x)
    num = int(tmp_num.group(0))
    print(num)
    
    # 設定爬取網址的路徑
    urls = [f'https://www.zapsplat.com/page/{i}/?s={i_kws}' for i in range(1, int(np.ceil(num/10)+1))]
    
    # 到此關鍵字下，每個要爬取的網址，抓取所需的資料
    for i_urls in urls:
        print('----- '+i_urls)
        a = session.get(i_urls)
        soup = BeautifulSoup(a.text, 'lxml')
        post_all = soup.find_all('div',{'class':'musicdiv fromshortcode'})
        posts = posts + src02.extract_post(post_all)
        
        # sleep
        time.sleep(random.uniform(1, 2))

laugh
233
----- https://www.zapsplat.com/page/1/?s=laugh
----- https://www.zapsplat.com/page/2/?s=laugh
----- https://www.zapsplat.com/page/3/?s=laugh
----- https://www.zapsplat.com/page/4/?s=laugh
----- https://www.zapsplat.com/page/5/?s=laugh
----- https://www.zapsplat.com/page/6/?s=laugh
----- https://www.zapsplat.com/page/7/?s=laugh
----- https://www.zapsplat.com/page/8/?s=laugh
----- https://www.zapsplat.com/page/9/?s=laugh
----- https://www.zapsplat.com/page/10/?s=laugh
----- https://www.zapsplat.com/page/11/?s=laugh
----- https://www.zapsplat.com/page/12/?s=laugh
----- https://www.zapsplat.com/page/13/?s=laugh
----- https://www.zapsplat.com/page/14/?s=laugh
----- https://www.zapsplat.com/page/15/?s=laugh
----- https://www.zapsplat.com/page/16/?s=laugh
----- https://www.zapsplat.com/page/17/?s=laugh
----- https://www.zapsplat.com/page/18/?s=laugh
----- https://www.zapsplat.com/page/19/?s=laugh
----- https://www.zapsplat.com/page/20/?s=laugh
----- https://www.zapsplat.com/page/21/

In [71]:
a.close()
session.close()

In [72]:
# unique result
uni_posts = []
for x in posts:
    if x not in uni_posts:
        uni_posts.append(x)

print('original # of posts:\t', len(posts))
print('unique # of posts:\t', len(uni_posts))

original # of posts:	 670
unique # of posts:	 650


In [73]:
# 抓出 mp3 的 webpage-link
mp3_link = [x['title']['link'] for x in uni_posts]

# 計算檔案個數
digits = int( math.log10(len(uni_posts)) ) + 1

# 檔案名稱, 存檔路徑, 下載連結
all_filename = ["s02-" + str(i).zfill(digits) + ".mp3" for i in range(0, len(uni_posts))]
all_path = [os.path.join(path_category_save, x) for x in all_filename]
all_url = [x['download_link'] for x in uni_posts]

#### download .mp3

In [74]:
%%time

csv_filename = []
csv_url = []
for i in range(len(uni_posts)):
    print(i, '\t', all_filename[i], '\t', mp3_link[i])
    src02.download_mp3(all_url[i], mp3_link[i], all_path[i])
    csv_filename.append(all_filename[i])
    csv_url.append(all_url[i])

0 	 s02-000.mp3 	 https://www.zapsplat.com/music/1-and-3-year-old-children-playing-with-pots-and-pans-on-kitchen-floor-laughing-and-having-fun/
1 	 s02-001.mp3 	 https://www.zapsplat.com/music/18-month-old-boy-single-laugh/
2 	 s02-002.mp3 	 https://www.zapsplat.com/music/18-month-old-boy-titter-style-laugh/
3 	 s02-003.mp3 	 https://www.zapsplat.com/music/20-month-old-toddler-laughing-being-tickled/
4 	 s02-004.mp3 	 https://www.zapsplat.com/music/audience-laughter-200-people-1-man-louder-then-everyone-else-komedia-comedy-club-comic-boom-uk/
5 	 s02-005.mp3 	 https://www.zapsplat.com/music/audience-laughter-200-people-komedia-comedy-club-comic-boom-uk-1/
6 	 s02-006.mp3 	 https://www.zapsplat.com/music/audience-laughter-200-people-komedia-comedy-club-comic-boom-uk-10/
7 	 s02-007.mp3 	 https://www.zapsplat.com/music/audience-laughter-200-people-komedia-comedy-club-comic-boom-uk-11/
8 	 s02-008.mp3 	 https://www.zapsplat.com/music/audience-laughter-200-people-komedia-comedy-club-comic-

88 	 s02-088.mp3 	 https://www.zapsplat.com/music/cartoon-monster-gremlin-troll-or-creature-crazy-laugh-1/
89 	 s02-089.mp3 	 https://www.zapsplat.com/music/cartoon-monster-gremlin-troll-or-creature-crazy-laugh-2/
90 	 s02-090.mp3 	 https://www.zapsplat.com/music/cartoon-mouse-laughter-1/
91 	 s02-091.mp3 	 https://www.zapsplat.com/music/cartoon-mouse-laughter-2/
92 	 s02-092.mp3 	 https://www.zapsplat.com/music/cartoon-silly-high-pitched-laugh/
93 	 s02-093.mp3 	 https://www.zapsplat.com/music/cartoon-voice-male-laughter-boss/
94 	 s02-094.mp3 	 https://www.zapsplat.com/music/cartoon-voice-male-laughter-gangsta/
95 	 s02-095.mp3 	 https://www.zapsplat.com/music/cartoon-wheezy-laugh-1/
96 	 s02-096.mp3 	 https://www.zapsplat.com/music/cartoon-wheezy-laugh-2/
97 	 s02-097.mp3 	 https://www.zapsplat.com/music/cartoon-wheezy-laugh-3/
98 	 s02-098.mp3 	 https://www.zapsplat.com/music/cartoon-witches-cackle-laugh-1/
99 	 s02-099.mp3 	 https://www.zapsplat.com/music/cartoon-witches-cackle-la

178 	 s02-178.mp3 	 https://www.zapsplat.com/music/laugh-male-ho-ho-ho-santa-clause-1/
179 	 s02-179.mp3 	 https://www.zapsplat.com/music/laugh-male-ho-ho-ho-santa-clause-2/
180 	 s02-180.mp3 	 https://www.zapsplat.com/music/laugh-male-slightly-evil-1/
181 	 s02-181.mp3 	 https://www.zapsplat.com/music/laugh-male-slightly-evil-2/
182 	 s02-182.mp3 	 https://www.zapsplat.com/music/male-funny-cartoon-laugh-1/
183 	 s02-183.mp3 	 https://www.zapsplat.com/music/male-funny-cartoon-laugh-2/
184 	 s02-184.mp3 	 https://www.zapsplat.com/music/male-funny-cartoon-laugh-3/
185 	 s02-185.mp3 	 https://www.zapsplat.com/music/male-funny-cartoon-laugh-sarcastic-unimpressed/
186 	 s02-186.mp3 	 https://www.zapsplat.com/music/male-funny-cartoon-laugh-snigger-sly/
187 	 s02-187.mp3 	 https://www.zapsplat.com/music/male-laugh-silly-and-exaggerated-1/
188 	 s02-188.mp3 	 https://www.zapsplat.com/music/male-laugh-silly-and-exaggerated-2/
189 	 s02-189.mp3 	 https://www.zapsplat.com/music/male-laugh-sly-mea

260 	 s02-260.mp3 	 https://www.zapsplat.com/music/cartoon-character-laugh-chuckle/
261 	 s02-261.mp3 	 https://www.zapsplat.com/music/cartoon-character-laugh-high-pitched/
262 	 s02-262.mp3 	 https://www.zapsplat.com/music/cartoon-character-laugh-male-revenge-1/
263 	 s02-263.mp3 	 https://www.zapsplat.com/music/cartoon-character-laugh-male-revenge-2/
264 	 s02-264.mp3 	 https://www.zapsplat.com/music/cartoon-character-laugh-sly-1/
265 	 s02-265.mp3 	 https://www.zapsplat.com/music/cartoon-character-laugh-sly-2/
266 	 s02-266.mp3 	 https://www.zapsplat.com/music/cartoon-duck-laugh/
267 	 s02-267.mp3 	 https://www.zapsplat.com/music/cartoon-high-pitched-laugh-1/
268 	 s02-268.mp3 	 https://www.zapsplat.com/music/cartoon-high-pitched-laugh-10/
269 	 s02-269.mp3 	 https://www.zapsplat.com/music/cartoon-high-pitched-laugh-11/
270 	 s02-270.mp3 	 https://www.zapsplat.com/music/cartoon-high-pitched-laugh-12/
271 	 s02-271.mp3 	 https://www.zapsplat.com/music/cartoon-high-pitched-laugh-13/
2

352 	 s02-352.mp3 	 https://www.zapsplat.com/music/male-laugh-snigger-with-mouth-closed-2/
353 	 s02-353.mp3 	 https://www.zapsplat.com/music/male-laugh-snigger-with-mouth-closed-3/
354 	 s02-354.mp3 	 https://www.zapsplat.com/music/male-adult-laugh-in-celebration-or-happy/
355 	 s02-355.mp3 	 https://www.zapsplat.com/music/male-crazy-laugh/
356 	 s02-356.mp3 	 https://www.zapsplat.com/music/middle-aged-female-laughing-slightly-wheezy-1/
357 	 s02-357.mp3 	 https://www.zapsplat.com/music/middle-aged-female-laughing-slightly-wheezy-2/
358 	 s02-358.mp3 	 https://www.zapsplat.com/music/monster-creature-evil-wheezy-laugh-exhale/
359 	 s02-359.mp3 	 https://www.zapsplat.com/music/restaurant-ambience-people-chat-and-laughter-speyer-germany/
360 	 s02-360.mp3 	 https://www.zapsplat.com/music/short-cute-young-female-laugh/
361 	 s02-361.mp3 	 https://www.zapsplat.com/music/small-group-of-20-zombies-yelling-and-laughing-loudly-long/
362 	 s02-362.mp3 	 https://www.zapsplat.com/music/small-mixe

427 	 s02-427.mp3 	 https://www.zapsplat.com/music/angry-female-scream-shout-ok/
428 	 s02-428.mp3 	 https://www.zapsplat.com/music/baby-girl-cute-scream-short/
429 	 s02-429.mp3 	 https://www.zapsplat.com/music/baby-girl-cute-laugh-excited-giggle/
430 	 s02-430.mp3 	 https://www.zapsplat.com/music/baby-monster-or-hatchling-scream-cry/
431 	 s02-431.mp3 	 https://www.zapsplat.com/music/baby-scream-and-cry-followed-by-whining-9-months-old/
432 	 s02-432.mp3 	 https://www.zapsplat.com/music/boom-crackle-and-scream-sampler-analo-fireworks-boom-glitch-granular-analog-distort-distortion-saturation-explode-glitchy-harsh-sci-fi/
433 	 s02-433.mp3 	 https://www.zapsplat.com/music/boy-3-years-old-whinge-scream-and-cry/
434 	 s02-434.mp3 	 https://www.zapsplat.com/music/cb-radio-male-voice-could-be-police-screaming-1/
435 	 s02-435.mp3 	 https://www.zapsplat.com/music/cb-radio-male-voice-could-be-police-screaming-2/
436 	 s02-436.mp3 	 https://www.zapsplat.com/music/cb-radio-male-voice-could-be-

514 	 s02-514.mp3 	 https://www.zapsplat.com/music/male-scream-horror-pain-02/
515 	 s02-515.mp3 	 https://www.zapsplat.com/music/male-scream-horror-pain-03/
516 	 s02-516.mp3 	 https://www.zapsplat.com/music/male-scream-horror-pain-04/
517 	 s02-517.mp3 	 https://www.zapsplat.com/music/male-scream-strain-trying-to-lift-heavy-object-1/
518 	 s02-518.mp3 	 https://www.zapsplat.com/music/male-scream-strain-trying-to-lift-heavy-object-2/
519 	 s02-519.mp3 	 https://www.zapsplat.com/music/male-scream-strain-trying-to-lift-heavy-object-3/
520 	 s02-520.mp3 	 https://www.zapsplat.com/music/male-scream-strain-trying-to-lift-heavy-object-4/
521 	 s02-521.mp3 	 https://www.zapsplat.com/music/male-scream-strain-trying-to-lift-heavy-object-5/
522 	 s02-522.mp3 	 https://www.zapsplat.com/music/male-man-scream/
523 	 s02-523.mp3 	 https://www.zapsplat.com/music/male-man-shout-scream-in-pain-1/
524 	 s02-524.mp3 	 https://www.zapsplat.com/music/male-man-shout-scream-in-pain-2/
525 	 s02-525.mp3 	 ht

604 	 s02-604.mp3 	 https://www.zapsplat.com/music/herd-of-pigs-grunting-and-screaming-madly-2/
605 	 s02-605.mp3 	 https://www.zapsplat.com/music/herd-of-pigs-grunting-and-screaming-madly-3/
606 	 s02-606.mp3 	 https://www.zapsplat.com/music/horror-bone-break-arm-or-leg-with-male-scream-1/
607 	 s02-607.mp3 	 https://www.zapsplat.com/music/horror-bone-break-arm-or-leg-with-male-scream-2/
608 	 s02-608.mp3 	 https://www.zapsplat.com/music/horror-bone-break-arm-or-leg-with-male-scream-3/
609 	 s02-609.mp3 	 https://www.zapsplat.com/music/horror-knife-stab-into-flesh-then-pull-out-very-squelchy-with-male-scream-1/
610 	 s02-610.mp3 	 https://www.zapsplat.com/music/horror-knife-stab-into-flesh-then-pull-out-very-squelchy-with-male-scream-2/
611 	 s02-611.mp3 	 https://www.zapsplat.com/music/horror-male-being-strangled-breaking-neck-with-screaming/
612 	 s02-612.mp3 	 https://www.zapsplat.com/music/large-crowd-panic-shouting-and-screaming-slight-distance/
613 	 s02-613.mp3 	 https://www.za

#### save metadata

In [75]:
# for csv
path_csv = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.csv')
data_csv = pd.DataFrame({
    'filename': csv_filename,
    'url': csv_url
})

# check exist?
if os.path.isfile(path_csv):
    ori_data_csv = pd.read_csv(path_csv)
    data_csv = pd.concat([ori_data_csv, data_csv], ignore_index=True).drop_duplicates()

# save
data_csv.to_csv(path_csv, encoding = 'utf-8', index = 0)

In [76]:
# for json
path_json = os.path.join(path_metadata[0], os.path.basename(path_category_save) + '.json')
res_uni2json = [{'filename':a, 'metadata':b} for a, b in zip(all_filename, uni_posts)]

# check exist?
if os.path.isfile(path_json):
    with open(path_json, encoding='utf-8') as f:
        tmp = json.load(f)
    res_uni2json = tmp + res_uni2json

# unique result
data_json = []
for x in res_uni2json:
    if x not in data_json:
        data_json.append(x)
# save
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(data_json, jsonfile, ensure_ascii = False)

## END

---

## archive